In [1]:
import pandas as pd
import numpy as np

In [2]:
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings', 'Kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
            'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2], 
            'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
            'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)

In [3]:
df.head()

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741


create a new column "Diff_score" where it's value should be difference of 'Scores' between consecutive rows.

In [4]:
tmp = []
for r1 in df.itertuples():
    for r2 in df.itertuples():
        if (r1[0] == 0) & (r2[0] == 0):
            tmp.append(np.nan)
        elif r1[0]+1 == r2[0]:
            tmp.append(r2[-1] - r1[-1])
tmp            

df['Diff_score'] = tmp
df.head()

,Team,Rank,Year,Points,Diff_score
0,Riders,1,2014,876,NaN
1,Riders,2,2015,789,-87.0
2,Devils,2,2014,863,74.0
3,Devils,3,2015,673,-190.0
4,Kings,3,2014,741,68.0


###### pandas groupby
    A group level operations involves 3 levels:
        - splitting the data
        - operation
        - combining results

    groupby supports 3 operations:
      - aggregation (min,max,sum,avg,count)
      - transformation
      - filter

In [5]:
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings', 'Kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
            'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2], 
            'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
            'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)
df.head()

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741


In [6]:
# df.Points.mean()

In [7]:
tmp = []
for group in df.Team.unique():
    t_df = df[df.Team == group] # spliting
    t_df = np.mean(t_df['Points']) # operation
    tmp.append(t_df) # combine result
    print(group, t_df)

Riders 762.25
Devils 768.0
Kings 774.25
Royals 752.5


###### aggregate operations

In [8]:
## compute average on Points for each team.
# df.groupby('Team').groups
df.groupby(['Team'])['Points'].mean()

Team
Devils    768.00
Kings     774.25
Riders    762.25
Royals    752.50
Name: Points, dtype: float64

In [9]:
df.groupby(['Team'])[['Points']].mean()

,Points
Team,
Devils,768.00
Kings,774.25
Riders,762.25
Royals,752.50


In [10]:
df.groupby(['Team'])[['Points']].mean().reset_index()

,Team,Points
0,Devils,768.00
1,Kings,774.25
2,Riders,762.25
3,Royals,752.50


In [11]:
df.groupby(['Team'])[['Points']].mean().reset_index().rename(columns = {'Points':"Avg_Points"})

,Team,Avg_Points
0,Devils,768.00
1,Kings,774.25
2,Riders,762.25
3,Royals,752.50


In [12]:
df.groupby(['Team'])[['Points']].max().reset_index().rename(columns = {'Points':"Max_Points"})

,Team,Max_Points
0,Devils,863
1,Kings,812
2,Riders,876
3,Royals,804


In [13]:
df.groupby(['Team'])[['Points']].max().reset_index().rename(columns = {'Points':"Max_Points"}).sort_values('Max_Points')

,Team,Max_Points
3,Royals,804
1,Kings,812
0,Devils,863
2,Riders,876


In [14]:
df.groupby(['Team']).mean()

,Rank,Year,Points
Team,,,
Devils,2.50,2014.5,768.00
Kings,2.25,2015.5,774.25
Riders,1.75,2015.5,762.25
Royals,2.50,2014.5,752.50


In [15]:
df.groupby(['Team'])[['Rank','Points']].mean().reset_index()

,Team,Rank,Points
0,Devils,2.50,768.00
1,Kings,2.25,774.25
2,Riders,1.75,762.25
3,Royals,2.50,752.50


In [16]:
## compute average on Points for each team-year.
# an example of splitting data on multiple columns
df.groupby(['Team','Year'])['Points'].mean()

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
Name: Points, dtype: int64

In [17]:
## compute maximum on Rank for each team.
df.groupby('Team')['Rank'].max()

Team
Devils    3
Kings     4
Riders    2
Royals    4
Name: Rank, dtype: int64

In [18]:
### compute maximum on 'Rank' and "Average" on "Points".
df.groupby('Team')[['Rank','Points']].agg({"Points":'mean',"Rank":"max"})

,Points,Rank
Team,,
Devils,768.00,3
Kings,774.25,4
Riders,762.25,2
Royals,752.50,4


In [19]:
# df.nlargest()

###### transformation operation
    - returned object (Series) length ~ length of orignal dataframe

In [20]:
df.head()

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741


In [21]:
## add a column where each row (i) value = 'Points' in row (i) - average(all 'Points' belonging to the Team for row (i))

In [22]:
## for example, 0th index
# value = 876 - df[df.Team == 'Riders']['Points'].mean()
876 - df[df.Team == 'Riders']['Points'].mean()

113.75

In [23]:
df['diff'] = df.groupby('Team')['Points'].transform(lambda x: x - np.mean(x)) #np.mean(x) is computed at group level
df.head()

,Team,Rank,Year,Points,diff
0,Riders,1,2014,876,113.75
1,Riders,2,2015,789,26.75
2,Devils,2,2014,863,95.00
3,Devils,3,2015,673,-95.00
4,Kings,3,2014,741,-33.25


In [24]:
df['team_avg'] = df.groupby('Team')['Points'].transform(lambda x: np.mean(x)) #np.mean(x) is computed at group level
df.head()

,Team,Rank,Year,Points,diff,team_avg
0,Riders,1,2014,876,113.75,762.25
1,Riders,2,2015,789,26.75,762.25
2,Devils,2,2014,863,95.00,768.00
3,Devils,3,2015,673,-95.00,768.00
4,Kings,3,2014,741,-33.25,774.25


###### filter 

In [25]:
df.head()

,Team,Rank,Year,Points,diff,team_avg
0,Riders,1,2014,876,113.75,762.25
1,Riders,2,2015,789,26.75,762.25
2,Devils,2,2014,863,95.00,768.00
3,Devils,3,2015,673,-95.00,768.00
4,Kings,3,2014,741,-33.25,774.25


In [26]:
### filter out "team" details which played for more than two years.
df.groupby('Team').filter(lambda x: len(x) > 2)

,Team,Rank,Year,Points,diff,team_avg
0,Riders,1,2014,876,113.75,762.25
1,Riders,2,2015,789,26.75,762.25
4,Kings,3,2014,741,-33.25,774.25
5,Kings,4,2015,812,37.75,774.25
6,Kings,1,2016,756,-18.25,774.25
7,Kings,1,2017,788,13.75,774.25
8,Riders,2,2016,694,-68.25,762.25
11,Riders,2,2017,690,-72.25,762.25


In [27]:
# df.groupby('Team')['Points'].filter(lambda x: len(x) > 2)

###### Pandas merge/join

In [28]:
## personal_info(customer_id(unique), name, mobile, email, address, reg_date)
## transaction_info (transaction_id(unique),transaction_date, amount, items, quanity, customer_id)

    type of joins
         - inner join
         - outer join => (left outer join, right outer join, full outer join)

    what happens when performed:
        inner join => common records 
        left outer join => common records + remaining records from left table
        right outer join => common records + remaining records from right table
        full outer join => common records + remaining records from both the tables

In [29]:
pd.merge

<function pandas.core.reshape.merge.merge(left, right, how: str = 'inner', on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, sort: bool = False, suffixes=('_x', '_y'), copy: bool = True, indicator: bool = False, validate=None) -> 'DataFrame'>

In [30]:
left_df = pd.DataFrame({ 'id':[1,2,3,4,5,11],
            'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung','Peter'],
            'subject_id':['sub1','sub2','sub4','sub6','sub5','sub11']}) 
right_df = pd.DataFrame( {'id':[1,2,3,4,5,10],
                'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty','jack'],
                'subject_id':['sub2','sub4','sub3','sub6','sub5','sub10']})

        Print left_df and right_Df
       1. Merge on id
       2. Merge on id & subject_id
       3. Perform left join on key = id 
       4. Perform right join on key = id
       5. Perform outer join on key = id
       6. Perform left join on keys = (id, subject_id)
       7. Perform right join on keys = (id, subject_id)
       8. Perform outer join on keys = (id, subject_id)


In [31]:
# 1. Merge on id
pd.merge(left=left_df, right=right_df, how='inner', on= ['id'])

,id,Name_x,subject_id_x,Name_y,subject_id_y
0,1,Alex,sub1,Billy,sub2
1,2,Amy,sub2,Brian,sub4
2,3,Allen,sub4,Bran,sub3
3,4,Alice,sub6,Bryce,sub6
4,5,Ayoung,sub5,Betty,sub5


In [32]:
left_df.merge(right_df, 'inner', ['id'])

,id,Name_x,subject_id_x,Name_y,subject_id_y
0,1,Alex,sub1,Billy,sub2
1,2,Amy,sub2,Brian,sub4
2,3,Allen,sub4,Bran,sub3
3,4,Alice,sub6,Bryce,sub6
4,5,Ayoung,sub5,Betty,sub5


In [33]:
# 2. Merge on id & subject_id
pd.merge(left=left_df, right=right_df, how='inner', on= ['id','subject_id'])

,id,Name_x,subject_id,Name_y
0,4,Alice,sub6,Bryce
1,5,Ayoung,sub5,Betty


In [34]:
# left_df => [(1,sub1),(2,sub2),(3,sub4),(4,sub6),(5,sub5),(11,sub11)]
# right_df => [(1,sub2),(2,sub4),(3,sub3),(4,sub6),(5,sub5),(10,sub10)]

In [35]:
# 3. Perform left join on key = id 
pd.merge(left=left_df, right=right_df, how='left', on= ['id'])

,id,Name_x,subject_id_x,Name_y,subject_id_y
0,1,Alex,sub1,Billy,sub2
1,2,Amy,sub2,Brian,sub4
2,3,Allen,sub4,Bran,sub3
3,4,Alice,sub6,Bryce,sub6
4,5,Ayoung,sub5,Betty,sub5
5,11,Peter,sub11,NaN,NaN


In [36]:
# 4. Perform right join on key = id 
pd.merge(left=left_df, right=right_df, how='right', on= ['id'])

,id,Name_x,subject_id_x,Name_y,subject_id_y
0,1,Alex,sub1,Billy,sub2
1,2,Amy,sub2,Brian,sub4
2,3,Allen,sub4,Bran,sub3
3,4,Alice,sub6,Bryce,sub6
4,5,Ayoung,sub5,Betty,sub5
5,10,NaN,NaN,jack,sub10


In [37]:
# 5. Perform full outer join on key = id 
pd.merge(left=left_df, right=right_df, how='outer', on= ['id'])

,id,Name_x,subject_id_x,Name_y,subject_id_y
0,1,Alex,sub1,Billy,sub2
1,2,Amy,sub2,Brian,sub4
2,3,Allen,sub4,Bran,sub3
3,4,Alice,sub6,Bryce,sub6
4,5,Ayoung,sub5,Betty,sub5
5,11,Peter,sub11,NaN,NaN
6,10,NaN,NaN,jack,sub10


In [38]:
# 6. Perform left outer join on key = ['id ','subject_id']
pd.merge(left=left_df, right=right_df, how='left', on= ['id','subject_id']) # [2+4 = 6]

,id,Name_x,subject_id,Name_y
0,1,Alex,sub1,NaN
1,2,Amy,sub2,NaN
2,3,Allen,sub4,NaN
3,4,Alice,sub6,Bryce
4,5,Ayoung,sub5,Betty
5,11,Peter,sub11,NaN


In [39]:
# 7. Perform right outer join on key = ['id ','subject_id']
pd.merge(left=left_df, right=right_df, how='right', on= ['id','subject_id']) # [2+4 = 6]

,id,Name_x,subject_id,Name_y
0,4,Alice,sub6,Bryce
1,5,Ayoung,sub5,Betty
2,1,NaN,sub2,Billy
3,2,NaN,sub4,Brian
4,3,NaN,sub3,Bran
5,10,NaN,sub10,jack


In [40]:
# 8. Perform left outer join on key = ['id ','subject_id']
pd.merge(left=left_df, right=right_df, how='outer', on= ['id','subject_id']) # # [4+2+4 = 10]

,id,Name_x,subject_id,Name_y
0,1,Alex,sub1,NaN
1,2,Amy,sub2,NaN
2,3,Allen,sub4,NaN
3,4,Alice,sub6,Bryce
4,5,Ayoung,sub5,Betty
5,11,Peter,sub11,NaN
6,1,NaN,sub2,Billy
7,2,NaN,sub4,Brian
8,3,NaN,sub3,Bran
9,10,NaN,sub10,jack
